In [129]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split # for preprocessing
from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.svm import LinearSVR, SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor

import warnings
warnings.filterwarnings(action='ignore')

In [130]:
data = pd.read_csv('groceries prices St Petersburg.csv')

In [131]:
data

,Магазин,Название товара,Цена,Дата
0,Пятерочка,чай\nчай,800р. \nза 1 кг,18 июля 2022
1,Пятерочка,масло растительное,130р. \nза 1 л,18 июля 2022
2,Пятерочка,яйцо\nяйца 10 шт,80р. \nза единицу,18 июля 2022
3,Пятерочка,молоко\nмолоко,80р. \nза 1 л,18 июля 2022
4,Пятерочка,рыба\nрыба,400р. \nза 1 кг,18 июля 2022
5,Пятерочка,мясо\nГовядина,550р. \nза 1 кг,18 июля 2022
6,Пятерочка,фрукты\nяблоки,125р. \nза 1 кг,18 июля 2022
7,Пятерочка,капуста\nкапуста,40р. \nза 1 кг,18 июля 2022
8,Пятерочка,картофель\nкартошка,65р. \nза 1 кг,18 июля 2022
9,Пятерочка,мука\nмука,40р. \nза 1 кг,18 июля 2022


In [132]:
data.info() # to learn more about data 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Магазин          50 non-null     object
 1   Название товара  50 non-null     object
 2   Цена             50 non-null     object
 3   Дата             50 non-null     object
dtypes: object(4)
memory usage: 1.7+ KB


# Preprocessing

In [133]:
# cleaning up two entrance of item in 1 cell
data["Название товара"] = data["Название товара"].str.split('\n').str[0]

# cleaning up "за 1 кг" and "за 1 л" since we aren't intreseted in that:
data["Цена"] = data["Цена"].str.split('\n').str[0]
# we also ain't need currency
data["Цена"] = data["Цена"].str.split('р').str[0]

data

,Магазин,Название товара,Цена,Дата
0,Пятерочка,чай,800,18 июля 2022
1,Пятерочка,масло растительное,130,18 июля 2022
2,Пятерочка,яйцо,80,18 июля 2022
3,Пятерочка,молоко,80,18 июля 2022
4,Пятерочка,рыба,400,18 июля 2022
5,Пятерочка,мясо,550,18 июля 2022
6,Пятерочка,фрукты,125,18 июля 2022
7,Пятерочка,капуста,40,18 июля 2022
8,Пятерочка,картофель,65,18 июля 2022
9,Пятерочка,мука,40,18 июля 2022


In [134]:
print(data.dtypes)

Магазин            object
Название товара    object
Цена               object
Дата               object
dtype: object


In [138]:
data["Магазин"] = data["Магазин"].astype(str)
data["Название товара"] = data["Название товара"].astype(str)
data["Цена"] = data["Цена"].astype(str).astype(int)
data["Дата"] = data["Дата"].astype(str)
print(data.dtypes)

Магазин            object
Название товара    object
Цена                int64
Дата               object
dtype: object


In [139]:
def preprocess_inputs(df):
    df = df.copy()
    
    # One-hot encoding
    
    for column in ['Магазин', 'Название товара', 'Дата']:
        df = onehot_encode(df, column)
    
    # Split df into X and y
    y = df['Цена']
    X = df.drop('Цена', axis=1)
    
    # Train-test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, shuffle=True, random_state=1)
    
    # Scale X
    scaler = StandardScaler()
    scaler.fit(X_train)
    X_train = pd.DataFrame(scaler.transform(X_train), index=X_train.index, columns=X_train.columns)
    X_test = pd.DataFrame(scaler.transform(X_test), index=X_test.index, columns=X_test.columns)
    
    return X_train, X_test, y_train, y_test

In [140]:
X_train, X_test, y_train, y_test = preprocess_inputs(data) # storing modified version on X

In [123]:
X_train

,Магазин_Ашан,Магазин_Дикси,Магазин_Лента,Магазин_Магнит,Магазин_Окей,Магазин_Перекресток,Магазин_Пятëрочка,Магазин_Пятерочка,Магазин_Пятерчка,Магазин_Рынок,...,Дата_15 мая 2022,Дата_16 июня 2022,Дата_16 мая 2022,Дата_17 мая 2022,Дата_18 июля 2022,Дата_18 мая 2022,Дата_23 мая 2022,Дата_26 апреля 2022,Дата_27 апреля 2022,Дата_28 мая 2022
42,-0.246183,5.830952,-0.359211,-0.306186,-0.171499,-0.246183,-0.171499,-1.089725,-0.171499,-0.171499,...,-0.171499,-0.171499,0.0,-0.171499,-0.588348,-0.171499,-0.171499,-0.171499,-0.171499,-0.171499
49,-0.246183,-0.171499,-0.359211,3.265986,-0.171499,-0.246183,-0.171499,-1.089725,-0.171499,-0.171499,...,-0.171499,-0.171499,0.0,-0.171499,-0.588348,-0.171499,-0.171499,-0.171499,-0.171499,-0.171499
26,-0.246183,-0.171499,-0.359211,-0.306186,-0.171499,-0.246183,-0.171499,0.917663,-0.171499,-0.171499,...,-0.171499,-0.171499,0.0,-0.171499,-0.588348,-0.171499,-0.171499,-0.171499,-0.171499,-0.171499
22,-0.246183,-0.171499,-0.359211,-0.306186,-0.171499,-0.246183,-0.171499,0.917663,-0.171499,-0.171499,...,-0.171499,-0.171499,0.0,-0.171499,-0.588348,-0.171499,-0.171499,-0.171499,-0.171499,-0.171499
13,-0.246183,-0.171499,2.783882,-0.306186,-0.171499,-0.246183,-0.171499,-1.089725,-0.171499,-0.171499,...,-0.171499,-0.171499,0.0,-0.171499,-0.588348,-0.171499,-0.171499,-0.171499,-0.171499,-0.171499
41,4.062019,-0.171499,-0.359211,-0.306186,-0.171499,-0.246183,-0.171499,-1.089725,-0.171499,-0.171499,...,-0.171499,-0.171499,0.0,-0.171499,-0.588348,-0.171499,-0.171499,-0.171499,-0.171499,-0.171499
17,-0.246183,-0.171499,-0.359211,-0.306186,-0.171499,-0.246183,-0.171499,0.917663,-0.171499,-0.171499,...,-0.171499,-0.171499,0.0,-0.171499,-0.588348,-0.171499,-0.171499,-0.171499,-0.171499,-0.171499
45,-0.246183,-0.171499,-0.359211,3.265986,-0.171499,-0.246183,-0.171499,-1.089725,-0.171499,-0.171499,...,-0.171499,-0.171499,0.0,-0.171499,-0.588348,-0.171499,-0.171499,-0.171499,-0.171499,-0.171499
24,-0.246183,-0.171499,-0.359211,-0.306186,-0.171499,-0.246183,-0.171499,0.917663,-0.171499,-0.171499,...,-0.171499,-0.171499,0.0,-0.171499,-0.588348,-0.171499,-0.171499,-0.171499,-0.171499,-0.171499
23,-0.246183,-0.171499,-0.359211,-0.306186,-0.171499,-0.246183,-0.171499,0.917663,-0.171499,-0.171499,...,-0.171499,-0.171499,0.0,-0.171499,-0.588348,-0.171499,-0.171499,-0.171499,-0.171499,-0.171499


In [141]:
X_train.mean() 

Магазин_Ашан                             -4.599495e-17
Магазин_Дикси                             1.015061e-16
Магазин_Лента                             9.198991e-17
Магазин_Магнит                           -7.295751e-17
Магазин_Окей                              0.000000e+00
Магазин_Перекресток                      -2.537653e-17
Магазин_Пятëрочка                         0.000000e+00
Магазин_Пятерочка                         1.141944e-16
Магазин_Пятерчка                          0.000000e+00
Магазин_Рынок                             0.000000e+00
Магазин_нонейм универсам                  0.000000e+00
Название товара_гречка                    0.000000e+00
Название товара_капуста                  -5.075305e-17
Название товара_картофель                 8.881784e-17
Название товара_кисломолочные продукты    0.000000e+00
Название товара_макароны                  1.015061e-16
Название товара_масло растительное       -2.061843e-17
Название товара_молоко                    1.268826e-17
Название т

In [142]:
y_train

42      50
49      79
26      55
22     600
13      40
41     200
17      79
45     500
24      70
23     450
4      400
33      50
14     500
30     150
10      15
28     329
44     324
34      92
18    1100
20      90
25      98
6      125
7       40
47      80
1      130
16      58
0      800
15     112
5      550
11      40
9       40
8       65
12    1200
43     596
37      77
Name: Цена, dtype: int64

# Training


In [145]:
models = {
    "                     Linear Regression": LinearRegression(),
    " Linear Regression (L2 Regularization)": Ridge(),
    " Linear Regression (L1 Regularization)": Lasso(),
    "                   K-Nearest Neighbors": KNeighborsRegressor(),
    "                        Neural Network": MLPRegressor(),
    "Support Vector Machine (Linear Kernel)": LinearSVR(),
    "   Support Vector Machine (RBF Kernel)": SVR(),
    "                         Decision Tree": DecisionTreeRegressor(),
    "                         Random Forest": RandomForestRegressor(),
    "                     Gradient Boosting": GradientBoostingRegressor(),
    "                               XGBoost": XGBRegressor(),
    "                              LightGBM": LGBMRegressor(),
    "                              CatBoost": CatBoostRegressor(verbose=0)
}

# training all our models

for name, model in models.items():
    model.fit(X_train, y_train)
    print(name + " trained.")

                     Linear Regression trained.
 Linear Regression (L2 Regularization) trained.
 Linear Regression (L1 Regularization) trained.
                   K-Nearest Neighbors trained.
                        Neural Network trained.
Support Vector Machine (Linear Kernel) trained.
   Support Vector Machine (RBF Kernel) trained.
                         Decision Tree trained.
                         Random Forest trained.
                     Gradient Boosting trained.
                               XGBoost trained.
                              LightGBM trained.
                              CatBoost trained.


# Results


In [146]:
for name, model in models.items():
    print(name + " R^2 Score: {:.5f}".format(model.score(X_test, y_test))) 

                     Linear Regression R^2 Score: -46244947296408164970400841728.00000
 Linear Regression (L2 Regularization) R^2 Score: 0.32998
 Linear Regression (L1 Regularization) R^2 Score: 0.21421
                   K-Nearest Neighbors R^2 Score: -0.67058
                        Neural Network R^2 Score: -0.47657
Support Vector Machine (Linear Kernel) R^2 Score: -0.44389
   Support Vector Machine (RBF Kernel) R^2 Score: -0.11419
                         Decision Tree R^2 Score: 0.22958
                         Random Forest R^2 Score: 0.00871
                     Gradient Boosting R^2 Score: 0.10632
                               XGBoost R^2 Score: -0.04642
                              LightGBM R^2 Score: -0.27182
                              CatBoost R^2 Score: 0.11957


# Outcome:
    
L2 Regularization is the winner amongst the other models, which don't show any remarkable results on the given dataset. It's either overfitted or underfitted. For that, let's see performance of the train data:

In [151]:
for name, model in models.items():
    print(name + " R^2 Score: {:.5f}".format(model.score(X_train, y_train))) 

                     Linear Regression R^2 Score: 0.80168
 Linear Regression (L2 Regularization) R^2 Score: 0.95289
 Linear Regression (L1 Regularization) R^2 Score: 0.95305
                   K-Nearest Neighbors R^2 Score: 0.31510
                        Neural Network R^2 Score: -0.48366
Support Vector Machine (Linear Kernel) R^2 Score: -0.57031
   Support Vector Machine (RBF Kernel) R^2 Score: -0.29665
                         Decision Tree R^2 Score: 1.00000
                         Random Forest R^2 Score: 0.93840
                     Gradient Boosting R^2 Score: 0.99675
                               XGBoost R^2 Score: 1.00000
                              LightGBM R^2 Score: 0.00000
                              CatBoost R^2 Score: 0.99802


.

it's not very bad of a performance, except several parameters. I assume my data is overfitted, or the dataser is not suffient to obtain a better performance. I will try to engineer my model to push the performance up. 

The obvious solution would be to feed more data, so this is where we go. Check code for Moscow price prediction model